# Working with Exasol using IBIS dataframe library.

In this notebook, we will show some basic operations on Exasol data using Ibis. You can find more detailed information on using Ibis on the official [Ibis project](https://ibis-project.org/) website.

The notebook is organized as a quickstart tutorial in which we will be looking at US flight delays. In particular, we will explore the delay caused by the carrier. We will rank the carriers using the delay as the performance metric. The data is publicly accessible at the [Bureau of Transportation Statistics](https://www.transtats.bts.gov/Homepage.asp) of the US Department of Transportation.

## Prerequisites

Prior to using this notebook the following steps need to be completed:
1. [Configure the AI-Lab](../main_config.ipynb).
2. [Load the US Flights data](../data/data_flights.ipynb).

## Setup

### Open Secure Configuration Storage

In [14]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

Output()

Box(children=(Box(children=(Label(value='Configuration Store', layout=Layout(border_bottom='solid 1px', border…

## connect

Let's connect to the Exasol database

In [3]:
import ibis

conn = ibis.exasol.connect(
    user=ai_lab_config.db_user,
    password=ai_lab_config.db_password,
    host=ai_lab_config.db_host_name,
    port=ai_lab_config.db_port,
    schema=ai_lab_config.db_schema,
    compression=True
)

# table

We will start by creating a `table` object for the table with the flight delay data.

In [5]:
flights = conn.table('US_FLIGHTS')

Let's have a look at the content of this table.

In [6]:
flights.head().to_pandas()

,FL_DATE,OP_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_STATE_ABR,DEST_AIRPORT_SEQ_ID,DEST_STATE_ABR,CRS_DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_DELAY,...,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2024-01-31,20397,1432105,ME,1127805,VA,0525,-10.00,0736,-32.00,...,None,False,131.00,109.00,482.00,None,None,None,None,None
1,2024-01-01,19977,1226603,TX,1129202,CO,1625,11.00,1801,15.00,...,None,False,156.00,160.00,862.00,5.00,0.00,4.00,0.00,6.00
2,2024-01-01,19977,1226603,TX,1129202,CO,1805,-1.00,1940,1.00,...,None,False,155.00,157.00,862.00,None,None,None,None,None
3,2024-01-01,19977,1226603,TX,1129202,CO,2021,127.00,2159,104.00,...,None,False,158.00,135.00,862.00,43.00,0.00,0.00,0.00,61.00
4,2024-01-31,19393,1082106,MD,1105703,NC,0920,-5.00,1105,-25.00,...,None,False,105.00,85.00,361.00,None,None,None,None,None


# filter

Should we compute the statistics on all records in the table?
What about canceled or diverted flights? How should we account for them? Let's see what information we've got for such unfortunate flights.

In [7]:
flights.filter(flights.CANCELLED).head().to_pandas()

,FL_DATE,OP_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_STATE_ABR,DEST_AIRPORT_SEQ_ID,DEST_STATE_ABR,CRS_DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_DELAY,...,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2024-01-01,20304,1027506,WA,1474703,WA,0500,None,0611,None,...,B,False,71.00,None,213.00,None,None,None,None,None
1,2024-01-01,20304,1071302,ID,1188402,WA,0800,-6.00,0816,None,...,B,False,76.00,None,287.00,None,None,None,None,None
2,2024-01-01,19930,1247805,NY,1477104,CA,2020,None,2359,None,...,A,False,399.00,None,2586.00,None,None,None,None,None
3,2024-01-01,19930,1477104,CA,1247805,NY,1038,None,1913,None,...,A,False,335.00,None,2586.00,None,None,None,None,None
4,2024-01-02,19393,1082106,MD,1320402,FL,1630,None,1855,None,...,A,False,145.00,None,787.00,None,None,None,None,None


In [8]:
flights.filter(flights.DIVERTED).head().to_pandas()

,FL_DATE,OP_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_STATE_ABR,DEST_AIRPORT_SEQ_ID,DEST_STATE_ABR,CRS_DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_DELAY,...,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2024-01-02,19805,1410702,AZ,1042302,TX,1330,-3.00,1643,None,...,None,True,133.00,None,872.00,None,None,None,None,None
1,2024-01-02,19930,1383002,HI,1474703,WA,1125,-5.00,1909,None,...,None,True,344.00,None,2640.00,None,None,None,None,None
2,2024-01-01,20368,1288904,NV,1188402,WA,0905,2.00,1134,None,...,None,True,149.00,None,806.00,None,None,None,None,None
3,2024-01-01,20304,1410702,AZ,1448903,OR,1518,-8.00,1702,None,...,None,True,164.00,None,893.00,None,None,None,None,None
4,2024-01-01,19977,1393008,IL,1029906,AK,1600,-7.00,1954,None,...,None,True,414.00,None,2846.00,None,None,None,None,None


There is no delay information for those flights. So, let's just exclude them.

# group_by and aggregate

Let's compute some statistics on the delay for each carrier.
We can chain together the `filter`, `group_by`, and `aggregate` operators.

In [9]:
delay_by_carrier = flights.filter((flights.CANCELLED | flights.DIVERTED).negate()).group_by('OP_CARRIER_AIRLINE_ID').aggregate(
    flights.CARRIER_DELAY.sum().name('combined_delay'),
    flights.CARRIER_DELAY.count().name('total_delayed'),
    flights.OP_CARRIER_AIRLINE_ID.count().name('total_flights')
)
delay_by_carrier.head().to_pandas()

,OP_CARRIER_AIRLINE_ID,combined_delay,total_delayed,total_flights
0,20397,100741.00,3924,15539
1,20416,89128.00,5274,20081
2,20436,87728.00,3814,14045
3,19977,283178.00,11659,54003
4,20398,66134.00,5473,19740


# mutate

Let's add two new columns to the statistics computed in the previous step - the percentage of flights that have been delayed and the average delay per flight.

In [10]:
delay_by_carrier = delay_by_carrier.mutate(
    percent_delayed=100 * delay_by_carrier.total_delayed / delay_by_carrier.total_flights,
    delay_per_flight=delay_by_carrier.combined_delay / delay_by_carrier.total_flights
)
delay_by_carrier.head().to_pandas()

,OP_CARRIER_AIRLINE_ID,combined_delay,total_delayed,total_flights,percent_delayed,delay_per_flight
0,20397,100741.00,3924,15539,25.252590,6.483107
1,20416,89128.00,5274,20081,26.263632,4.438424
2,20436,87728.00,3814,14045,27.155571,6.246209
3,19977,283178.00,11659,54003,21.589541,5.243746
4,20398,66134.00,5473,19740,27.725431,3.350253


# join

Now, let's link the table with the carrier names, that are stored in another table called US_AIRLINES.

In [11]:
airlines = conn.table('US_AIRLINES')
delay_by_carrier = delay_by_carrier.join(airlines, 'OP_CARRIER_AIRLINE_ID', 'inner')
delay_by_carrier.head().to_pandas()

,OP_CARRIER_AIRLINE_ID,combined_delay,total_delayed,total_flights,percent_delayed,delay_per_flight,CARRIER_NAME
0,19393,379809.00,26310,111613,23.572523,3.402910,Southwest Airlines Co. WN
1,19805,610041.00,22292,75889,29.374481,8.038596,American Airlines Inc. AA
2,19977,283178.00,11659,54003,21.589541,5.243746,United Air Lines Inc. UA
3,20397,100741.00,3924,15539,25.252590,6.483107,PSA Airlines Inc. OH
4,20304,581854.00,13082,54206,24.133860,10.734125,SkyWest Airlines Inc. OO


# order

Let's order the airlines from worst to best

In [12]:
delay_by_carrier = delay_by_carrier.order_by(delay_by_carrier.delay_per_flight.desc())
delay_by_carrier.head().to_pandas()

,OP_CARRIER_AIRLINE_ID,combined_delay,total_delayed,total_flights,percent_delayed,delay_per_flight,CARRIER_NAME
0,20304,581854.00,13082,54206,24.133860,10.734125,SkyWest Airlines Inc. OO
1,20409,173848.00,5565,19172,29.026706,9.067807,JetBlue Airways B6
2,19805,610041.00,22292,75889,29.374481,8.038596,American Airlines Inc. AA
3,20397,100741.00,3924,15539,25.252590,6.483107,PSA Airlines Inc. OH
4,20436,87728.00,3814,14045,27.155571,6.246209,Frontier Airlines Inc. F9


# select

Finally, we will select the columns to display and print out the 10 worst airlines.

In [13]:
delay_by_carrier = delay_by_carrier.select('CARRIER_NAME', 'percent_delayed', 'delay_per_flight')
delay_by_carrier.head(10).to_pandas()

,CARRIER_NAME,percent_delayed,delay_per_flight
0,SkyWest Airlines Inc. OO,24.133860,10.734125
1,JetBlue Airways B6,29.026706,9.067807
2,American Airlines Inc. AA,29.374481,8.038596
3,PSA Airlines Inc. OH,25.252590,6.483107
4,Frontier Airlines Inc. F9,27.155571,6.246209
5,Allegiant Air G4,22.340426,6.218025
6,Delta Air Lines Inc. DL,18.895309,5.742696
7,Endeavor Air Inc. 9E,23.250509,5.578087
8,Hawaiian Airlines Inc. HA,26.841699,5.336216
9,United Air Lines Inc. UA,21.589541,5.243746
